In [ ]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: SkiaSharp"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.TextToImage;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

var settings = Settings.LoadFromFile("config/settings.json");
var builder = Kernel.CreateBuilder();

builder
    .AddAzureOpenAIChatCompletion("gpt4o", settings.azureEndpoint, settings.apiKey);

var kernel = builder.Build();
var chatGPT = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
public static ImageContent GetImageContent(string path)
{
        var imageData = new ReadOnlyMemory<byte>(File.ReadAllBytes(path));
        ImageContent imageContent = new(new BinaryData(imageData), "image/png");
        return imageContent; 
}

In [ ]:
string systemPrompt = """
You are a helpfull assistant.
You get two images one with the object to find in the other.
""";

// Add system message
var chatHistory = new ChatHistory(systemPrompt);

var small = GetImageContent("data/rect.png");
var big = GetImageContent("data/more.png");

chatHistory.AddUserMessage(new ChatMessageContentItemCollection
{
    small,
    big
});

In [ ]:
var reply = await chatGPT.GetChatMessageContentAsync(chatHistory);
Console.WriteLine($"{reply}");